# How to use Amazon Forecast

Helps advanced users start with Amazon Forecast quickly. The demo notebook runs through a typical end to end usecase for a simple timeseries forecasting scenario. 

Prerequisites: 
[AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/installing.html) . 

For more informations about APIs, please check the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/what-is-forecast.html)

## Table Of Contents
* [Setting up](#setup)
* [Test Setup - Running first API](#hello)
* [Forecasting Example with Amazon Forecast](#forecastingExample)

**Read Every Cell FULLY before executing it**


## Setup<a class="anchor" id="setup"></a>

In [1]:
import boto3
from time import sleep
from util.fcst_utils import *
import subprocess

In [2]:
session = boto3.Session(region_name='us-west-2') 

forecast = session.client(service_name='forecast') 
forecastquery = session.client(service_name='forecastquery')

In [3]:
forecast.list_dataset_groups()

{'DatasetGroups': [{'DatasetGroupArn': 'arn:aws:forecast:us-west-2:126894715080:dataset-group/util_power_forecastdemo_dsg',
   'DatasetGroupName': 'util_power_forecastdemo_dsg',
   'CreationTime': datetime.datetime(2019, 11, 15, 17, 21, 3, 221000, tzinfo=tzlocal()),
   'LastModificationTime': datetime.datetime(2019, 11, 15, 17, 21, 14, 606000, tzinfo=tzlocal())},
  {'DatasetGroupArn': 'arn:aws:forecast:us-west-2:126894715080:dataset-group/testdatasetgroup',
   'DatasetGroupName': 'testdatasetgroup',
   'CreationTime': datetime.datetime(2019, 10, 25, 13, 57, 42, 177000, tzinfo=tzlocal()),
   'LastModificationTime': datetime.datetime(2019, 10, 25, 14, 1, 30, 764000, tzinfo=tzlocal())},
  {'DatasetGroupArn': 'arn:aws:forecast:us-west-2:126894715080:dataset-group/test',
   'DatasetGroupName': 'test',
   'CreationTime': datetime.datetime(2019, 10, 25, 13, 55, 49, 244000, tzinfo=tzlocal()),
   'LastModificationTime': datetime.datetime(2019, 10, 25, 13, 56, 6, 605000, tzinfo=tzlocal())}],
 'R

## Forecasting with Amazon Forecast<a class="anchor" id="forecastingExample"></a>
### Preparing your Data

In Amazon Forecast , a dataset is a collection of file(s) which contain data that is relevant for a forecasting task. A dataset must conform to a schema provided by Amazon Forecast. 

For this exercise, we use the individual household electric power consumption dataset. (Dua, D. and Karra Taniskidou, E. (2017). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.) We aggregate the usage data hourly. 

### Data Type

Amazon forecast can import data from Amazon S3. We first explore the data locally to see the fields

In [10]:
import pandas as pd
df = pd.read_csv("../data/item-demand-time.csv", dtype = object)
df.head(3)

,2014-01-01 01:00:00,38.34991708126038,client_12
0,2014-01-01 02:00:00,33.5820895522388,client_12
1,2014-01-01 03:00:00,34.41127694859037,client_12
2,2014-01-01 04:00:00,39.800995024875625,client_12


Now upload the data to S3. But before doing that, go into your AWS Console, select S3 for the service and create a new bucket inside the `Oregon` or `us-west-2` region. Use that bucket name convention of `amazon-forecast-unique-value-data`. The name must be unique, if you get an error, just adjust until your name works, then update the `bucketName` cell below.

In [11]:
s3 = session.client('s3')

In [12]:
accountId = boto3.client('sts').get_caller_identity().get('Account')

In [15]:
bucketName = "amazon-forecast-data-{0}".format(accountId) # Update the unique-value bit here.
key="elec_data/item-demand-time.csv"

In [16]:
s3.upload_file(Filename="../data/item-demand-time.csv", Bucket=bucketName, Key=key)

In [17]:
bucketName

'aws-seer-ruinon-test'

In [18]:
# One time setup only, uncomment the following command to create the role to provide to Amazon Forecast. 
# Save the generated role for all future calls to use for importing or exporting data. 

cmd = 'python ../setup_forecast_permissions.py '+bucketName
p = subprocess.Popen(cmd.split(' '), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [19]:
roleArn = 'arn:aws:iam::%s:role/amazonforecast'%accountId

### CreateDataset

More details about `Domain` and dataset type can be found on the [documentation](https://docs.aws.amazon.com/forecast/latest/dg/howitworks-domains-ds-types.html) . For this example, we are using [CUSTOM](https://docs.aws.amazon.com/forecast/latest/dg/custom-domain.html) domain with 3 required attributes `timestamp`, `target_value` and `item_id`. Also for your project name, update it to reflect your name in a lowercase format.

In [20]:
DATASET_FREQUENCY = "H" 
TIMESTAMP_FORMAT = "yyyy-MM-dd hh:mm:ss"

In [21]:
project = 'electric_power_forecastdemo' # Replace this with a unique name here, make sure the entire name is < 30 characters.
datasetName= project+'_ds'
datasetGroupName= project +'_gp'
s3DataPath = "s3://"+bucketName+"/"+key

In [22]:
datasetName

'electric_power_forecastdemo_ds'

### Schema Definition 
### We are defining the attributes for the model 

In [23]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }
   ]
}

response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = schema
                   )
datasetArn = response['DatasetArn']

In [24]:
forecast.describe_dataset(DatasetArn=datasetArn)

{'DatasetArn': 'arn:aws:forecast:us-west-2:126894715080:dataset/electric_power_forecastdemo_ds',
 'DatasetName': 'electric_power_forecastdemo_ds',
 'Domain': 'CUSTOM',
 'DatasetType': 'TARGET_TIME_SERIES',
 'DataFrequency': 'H',
 'Schema': {'Attributes': [{'AttributeName': 'timestamp',
    'AttributeType': 'timestamp'},
   {'AttributeName': 'target_value', 'AttributeType': 'float'},
   {'AttributeName': 'item_id', 'AttributeType': 'string'}]},
 'EncryptionConfig': {},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2019, 11, 18, 10, 14, 10, 961000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2019, 11, 18, 10, 14, 10, 961000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '165174e6-d3b7-41d3-98cc-206c977ebae6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 18 Nov 2019 18:14:11 GMT',
   'x-amzn-requestid': '165174e6-d3b7-41d3-98cc-206c977ebae6',
   'content-length': '521',
   'connection': 'keep-

In [25]:
create_dataset_group_response = forecast.create_dataset_group(DatasetGroupName=datasetGroupName,
                                                              Domain="CUSTOM",
                                                              DatasetArns= [datasetArn]
                                                             )
datasetGroupArn = create_dataset_group_response['DatasetGroupArn']

If you have an existing datasetgroup, you can update it using **update_dataset_group** to update dataset group.



In [26]:
forecast.describe_dataset_group(DatasetGroupArn=datasetGroupArn)

{'DatasetGroupName': 'electric_power_forecastdemo_gp',
 'DatasetGroupArn': 'arn:aws:forecast:us-west-2:126894715080:dataset-group/electric_power_forecastdemo_gp',
 'DatasetArns': ['arn:aws:forecast:us-west-2:126894715080:dataset/electric_power_forecastdemo_ds'],
 'Domain': 'CUSTOM',
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2019, 11, 18, 10, 14, 13, 987000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2019, 11, 18, 10, 14, 13, 987000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '0f5d2142-80f8-4333-84a8-9721cbf0f043',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Mon, 18 Nov 2019 18:14:15 GMT',
   'x-amzn-requestid': '0f5d2142-80f8-4333-84a8-9721cbf0f043',
   'content-length': '363',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### Create Data Import Job
Brings the data into Amazon Forecast system ready to forecast from raw data. 

In [27]:
datasetImportJobName = 'EP_DSIMPORT_JOB_TARGET'
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":s3DataPath,
                                                                 "RoleArn": roleArn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [28]:
ds_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(ds_import_job_arn)

arn:aws:forecast:us-west-2:126894715080:dataset-import-job/electric_power_forecastdemo_ds/EP_DSIMPORT_JOB_TARGET


Check the status of dataset, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on the data size. It can take 10 mins to be **ACTIVE**. This process will take 5 to 10 minutes.

In [29]:
while True:
    dataImportStatus = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)['Status']
    print(dataImportStatus)
    if dataImportStatus != 'ACTIVE' and dataImportStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
ACTIVE


In [30]:
forecast.describe_dataset_import_job(DatasetImportJobArn=ds_import_job_arn)

{'DatasetImportJobName': 'EP_DSIMPORT_JOB_TARGET',
 'DatasetImportJobArn': 'arn:aws:forecast:us-west-2:126894715080:dataset-import-job/electric_power_forecastdemo_ds/EP_DSIMPORT_JOB_TARGET',
 'DatasetArn': 'arn:aws:forecast:us-west-2:126894715080:dataset/electric_power_forecastdemo_ds',
 'TimestampFormat': 'yyyy-MM-dd hh:mm:ss',
 'DataSource': {'S3Config': {'Path': 's3://aws-seer-ruinon-test/elec_data/item-demand-time.csv',
   'RoleArn': 'arn:aws:iam::126894715080:role/ForecastRole'}},
 'FieldStatistics': {'item_id': {'Count': 23973,
   'CountDistinct': 3,
   'CountNull': 0},
  'target_value': {'Count': 23973,
   'CountDistinct': 4818,
   'CountNull': 0,
   'CountNan': 0,
   'Min': '0.0',
   'Max': '212.27197346600326',
   'Avg': 50.447323170680725,
   'Stddev': 38.72169238224658},
  'timestamp': {'Count': 23973,
   'CountDistinct': 7991,
   'CountNull': 0,
   'Min': '2014-01-01T01:00:00Z',
   'Max': '2014-11-29T23:00:00Z'}},
 'DataSize': 0.0010688817128539085,
 'Status': 'ACTIVE',
 'C

### Create Predictor with customer forecast horizon

Forecast horizon is the number of number of time points to predicted in the future. For weekly data, a value of 12 means 12 weeks. Our example is hourly data, we try forecast the next day, so we can set to 24.

In [31]:
predictorName= project+'_deepAr_algo'

In [32]:
forecastHorizon = 24

In [33]:
algorithmArn = 'arn:aws:forecast:::algorithm/Deep_AR_Plus'

In [34]:
create_predictor_response=forecast.create_predictor(PredictorName=predictorName, 
                                                  AlgorithmArn=algorithmArn,
                                                  ForecastHorizon=forecastHorizon,
                                                  PerformAutoML= False,
                                                  PerformHPO=False,
                                                  EvaluationParameters= {"NumberOfBacktestWindows": 1, 
                                                                         "BackTestWindowOffset": 24}, 
                                                  InputDataConfig= {"DatasetGroupArn": datasetGroupArn},
                                                  FeaturizationConfig= {"ForecastFrequency": "H", 
                                                                        "Featurizations": 
                                                                        [
                                                                          {"AttributeName": "target_value", 
                                                                           "FeaturizationPipeline": 
                                                                            [
                                                                              {"FeaturizationMethodName": "filling", 
                                                                               "FeaturizationMethodParameters": 
                                                                                {"frontfill": "none", 
                                                                                 "middlefill": "zero", 
                                                                                 "backfill": "zero"}
                                                                              }
                                                                            ]
                                                                          }
                                                                        ]
                                                                       }
                                                 )

In [35]:
predictorArn=create_predictor_response['PredictorArn']

Check the status of the predictor. When the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [36]:
while True:
    predictorStatus = forecast.describe_predictor(PredictorArn=predictorArn)['Status']
    print(predictorStatus)
    if predictorStatus != 'ACTIVE' and predictorStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRE

### Get Error Metrics

In [37]:
forecast.get_accuracy_metrics(PredictorArn=predictorArn)

{'PredictorEvaluationResults': [{'AlgorithmArn': 'arn:aws:forecast:::algorithm/Deep_AR_Plus',
   'TestWindows': [{'EvaluationType': 'SUMMARY',
     'Metrics': {'RMSE': 21.858090454389654,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.0971399005978474},
       {'Quantile': 0.5, 'LossValue': 0.285458860298834},
       {'Quantile': 0.1, 'LossValue': 0.2672397301542571}]}},
    {'TestWindowStart': datetime.datetime(2014, 11, 28, 16, 0, tzinfo=tzlocal()),
     'TestWindowEnd': datetime.datetime(2014, 11, 29, 16, 0, tzinfo=tzlocal()),
     'ItemCount': 3,
     'EvaluationType': 'COMPUTED',
     'Metrics': {'RMSE': 21.858090454389654,
      'WeightedQuantileLosses': [{'Quantile': 0.9,
        'LossValue': 0.0971399005978474},
       {'Quantile': 0.5, 'LossValue': 0.285458860298834},
       {'Quantile': 0.1, 'LossValue': 0.2672397301542571}]}}]}],
 'ResponseMetadata': {'RequestId': 'b3e5309f-fc35-4fb2-b731-75f5559ad0c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'c

### Create Forecast

Now create a forecast using the model that was trained.

In [38]:
forecastName= project+'_deepAR_algo_forecast'

In [39]:
create_forecast_response=forecast.create_forecast(ForecastName=forecastName,
                                                  PredictorArn=predictorArn)
forecastArn = create_forecast_response['ForecastArn']

Check the status of the forecast process, when the status change from **CREATE_IN_PROGRESS** to **ACTIVE**, we can continue to next steps. Depending on data size, model selection and hyper parameters，it can take 10 mins to more than one hour to be **ACTIVE**.

In [40]:
while True:
    forecastStatus = forecast.describe_forecast(ForecastArn=forecastArn)['Status']
    print(forecastStatus)
    if forecastStatus != 'ACTIVE' and forecastStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRE

### Get Forecast

Once created, the forecast results are ready and you view them. 

In [41]:
print(forecastArn)
forecastResponse = forecastquery.query_forecast(
    ForecastArn=forecastArn,
    Filters={"item_id":"client_12"}
)
print(forecastResponse)

arn:aws:forecast:us-west-2:126894715080:forecast/electric_power_forecastdemo_deepAR_algo_forecast
{'Forecast': {'Predictions': {'p10': [{'Timestamp': '2014-11-30T00:00:00', 'Value': 38.68849563598633}, {'Timestamp': '2014-11-30T01:00:00', 'Value': 37.88291549682617}, {'Timestamp': '2014-11-30T02:00:00', 'Value': 35.081077575683594}, {'Timestamp': '2014-11-30T03:00:00', 'Value': 32.3565673828125}, {'Timestamp': '2014-11-30T04:00:00', 'Value': 36.040401458740234}, {'Timestamp': '2014-11-30T05:00:00', 'Value': 35.27180099487305}, {'Timestamp': '2014-11-30T06:00:00', 'Value': 31.788686752319336}, {'Timestamp': '2014-11-30T07:00:00', 'Value': 31.290996551513672}, {'Timestamp': '2014-11-30T08:00:00', 'Value': 24.095046997070312}, {'Timestamp': '2014-11-30T09:00:00', 'Value': 25.74273681640625}, {'Timestamp': '2014-11-30T10:00:00', 'Value': 22.67711639404297}, {'Timestamp': '2014-11-30T11:00:00', 'Value': 19.87714385986328}, {'Timestamp': '2014-11-30T12:00:00', 'Value': 25.436525344848633}, {

# Export Forecast

You can export forecast to s3 bucket. To do so an role with s3 put access is needed, but this has already been created.

In [42]:
forecastExportName= project+'_deepAR_algo_forecast_export'

In [43]:
outputPath="s3://"+bucketName+"/output"

In [44]:
forecast_export_response = forecast.create_forecast_export_job(
                                                                ForecastExportJobName = forecastExportName,
                                                                ForecastArn=forecastArn, 
                                                                Destination = {
                                                                   "S3Config" : {
                                                                       "Path":outputPath,
                                                                       "RoleArn": roleArn
                                                                   } 
                                                                }
                                                              )

In [45]:
forecastExportJobArn = forecast_export_response['ForecastExportJobArn']
print(forecastExportJobArn)

arn:aws:forecast:us-west-2:126894715080:forecast-export-job/electric_power_forecastdemo_deepAR_algo_forecast/electric_power_forecastdemo_deepAR_algo_forecast_export


In [46]:
while True:
    forecastExportStatus = forecast.describe_forecast_export_job(ForecastExportJobArn=forecastExportJobArn)['Status']
    print(forecastExportStatus)
    if forecastExportStatus != 'ACTIVE' and forecastExportStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
ACTIVE


Check s3 bucket for results

In [47]:
s3.list_objects(Bucket=bucketName,Prefix="output")

{'ResponseMetadata': {'RequestId': '66451F38319A345C',
  'HostId': 'QT8HR4HXJMY3JrWWL3LBxTOoit7pvK0HI9g3NwMm8iPD1jrbcggXOU19qs9Ihu1oBZ+8vfwGShE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'QT8HR4HXJMY3JrWWL3LBxTOoit7pvK0HI9g3NwMm8iPD1jrbcggXOU19qs9Ihu1oBZ+8vfwGShE=',
   'x-amz-request-id': '66451F38319A345C',
   'date': 'Mon, 18 Nov 2019 19:14:40 GMT',
   'x-amz-bucket-region': 'us-west-2',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'output/_CHECK',
   'LastModified': datetime.datetime(2019, 11, 18, 19, 12, 35, tzinfo=tzutc()),
   'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'Size': 0,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'ruinon+scm',
    'ID': '865591f95fcf087ee758043856710c43539f97ab11d2631f00f329b72e88ce85'}},
  {'Key': 'output/_SUCCESS',
   'LastModified': datetime.datetime(2019, 11, 18, 19, 13, 49, tzi

# Cleanup

While Forecast is in preview there are no charges for using it, but to future proof this work below are the instructions to cleanup your work space.

In [48]:
# Delete forecast export for algorithms
wait_till_delete(lambda: forecast.delete_forecast_export_job(ForecastExportJobArn = forecastExportJobArn))

In [49]:
# Delete forecast
wait_till_delete(lambda: forecast.delete_forecast(ForecastArn = forecastArn))

In [50]:
# Delete predictor
wait_till_delete(lambda: forecast.delete_predictor(PredictorArn = predictorArn))

In [51]:
# Delete Import
wait_till_delete(lambda: forecast.delete_dataset_import_job(DatasetImportJobArn=ds_import_job_arn))

In [52]:
# Delete datasets
wait_till_delete(lambda: forecast.delete_dataset(DatasetArn=datasetArn))

In [53]:
# Delete Dataset Group
wait_till_delete(lambda: forecast.delete_dataset_group(DatasetGroupArn=datasetGroupArn))